In [1]:
!pip install geocoder


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import cv2
import numpy as np
import geocoder
import os
import time

# Reading label name from obj.names file
class_name = ['damaged']
with open(os.path.join("Downloads", r"C:\Users\Sravanthi\Downloads\object.names.txt"), 'r') as f:
    class_name = [cname.strip() for cname in f.readlines()]

# Importing model weights and config file
# Defining the model parameters
net1 = cv2.dnn.readNet(r'C:\Users\Sravanthi\Downloads\yolov4-tiny.weights', r'C:\Users\Sravanthi\Downloads\yolov4-tiny.cfg')
net1.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net1.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)
model1 = cv2.dnn_DetectionModel(net1)
model1.setInputParams(size=(416, 416), scale=1/255, swapRB=True)  # Adjust input size based on the YOLOv4-tiny configuration

# Defining the video source (0 for camera or file name for video)
cap = cv2.VideoCapture(r"C:\Users\Sravanthi\Downloads\WhatsApp Video 2023-09-11 at 15.24.53.mp4")
width = cap.get(3)
height = cap.get(4)
result = cv2.VideoWriter(r"C:\Users\Sravanthi\Downloads\result.avi",
                        cv2.VideoWriter_fourcc(*'MJPG'),
                        10, (416, 416))  # Adjusted width and height

# Defining parameters for result saving and get coordinates
# Defining initial values for some parameters in the script
g = geocoder.ip('me')
result_path = "bike_coordinates"
starting_time = time.time()
Conf_threshold = 0.5
NMS_threshold = 0.4
frame_counter = 0
i = 0
b = 0
skip_frames = 3  # Process every 3rd frame

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    #create a copy of the frame for displaying the original video
    original_frame = frame.copy()
    
    # Convert the frame to LAB color space
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)

    # Split the LAB image into L, A, and B channels
    l, a, b = cv2.split(lab)

    # Apply color thresholding based on the A and B channels
    # Adjust the threshold values to capture the damaged regions based on color
    color_mask = cv2.inRange(a, 100, 200) & cv2.inRange(b, 100, 200)

    # Apply texture analysis using the L channel
    # Apply a Gaussian filter to smooth the image
    l_blurred = cv2.GaussianBlur(l, (5, 5), 0)

    # Compute the gradient magnitude using the Sobel operator on the smoothed L channel
    gradient_x = cv2.Sobel(l_blurred, cv2.CV_64F, 1, 0, ksize=3)
    gradient_y = cv2.Sobel(l_blurred, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = np.sqrt(gradient_x*2 + gradient_y*2)

    # Normalize the gradient magnitude to the range [0, 255]
    gradient_magnitude = cv2.normalize(gradient_magnitude, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    # Apply texture thresholding based on the gradient magnitude
    # Adjust the threshold value to capture the damaged regions based on texture
    texture_mask = cv2.inRange(gradient_magnitude, 50, 255)

    # Combine the color and texture masks
    mask = cv2.bitwise_and(color_mask, texture_mask)

    # Find contours in the mask to detect damaged areas
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Set the threshold area to filter out small or large areas
    threshold_area = 1000  # Adjust this value based on your requirements

    # Calculate the total area of the bike
    bike_area = cv2.countNonZero(mask)

    # Iterate over the contours and identify the damaged areas
    damage_area = 0
    for contour in contours:
        # Filter contours based on area or shape properties to detect damaged areas
        area = cv2.contourArea(contour)
        if area > threshold_area:
            # Find the bounding box of the damaged area
            x, y, w, h = cv2.boundingRect(contour)

            # Draw a rectangle around the damaged area
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Add the name of the damaged area near its rectangle
            cv2.putText(frame, "Damaged", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)

            # Increment the damage area
            damage_area += area

    # Calculate the percentage of damage
    damage_percentage = (damage_area / bike_area) * 100

    # Draw the text for the damage percentage on top of the frame
    cv2.putText(frame, f"Damage: {damage_percentage:.2f}%", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
    concatenated_frames = np.hstack((original_frame,frame))

    # Display the frame with the detected damaged areas and the damage percentage
    cv2.imshow("Video Frame | original vs Damaged", concatenated_frames)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Status code Unknown from http://ipinfo.io/json: ERROR - HTTPConnectionPool(host='ipinfo.io', port=80): Max retries exceeded with url: /json (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000016DF22184C0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
C:\Users\Sravanthi\AppData\Local\Temp\ipykernel_5772\1377923007.py:64: RuntimeWarning: invalid value encountered in sqrt
  gradient_magnitude = np.sqrt(gradient_x*2 + gradient_y*2)


In [1]:
import cv2
import os
import numpy as np

# Load the bike image
original_image = cv2.imread(r"C:\\Users\\Sravanthi\\Downloads\\123.jpeg")
image = original_image.copy()

# Reading label names from obj.names file
with open(os.path.join("Downloads", r"C:\Users\Sravanthi\Downloads\object.names.txt"), 'r') as f:
    classes = f.read().splitlines()

# Importing model weights and config file for object detection
net = cv2.dnn.readNet(r'C:\Users\Sravanthi\Downloads\yolov4-tiny.weights', r'C:\Users\Sravanthi\Downloads\yolov4-tiny.cfg')
model = cv2.dnn_DetectionModel(net)
model.setInputParams(scale=1 / 255, size=(416, 416), swapRB=True)

# Detect objects in the image
classIds, scores, boxes = model.detect(original_image, confThreshold=0.6, nmsThreshold=0.4)

# Initialize variables for bike damage detection
damage_area = 0
bike_area = 0

# Iterate over the detected objects
for (classId, score, box) in zip(classIds, scores, boxes):
    if classes[int(classId[0])] == "damaged":
        # Draw bounding box for damaged object
        cv2.rectangle(image, (box[0], box[1]), (box[0] + box[2], box[1] + box[3]), color=(0, 255, 0), thickness=2)

        # Update bike area
        bike_area += (box[2] * box[3])

        # Crop the damaged region from the image
        damaged_region = original_image[int(box[1]):int(box[1] + box[3]), int(box[0]):int(box[0] + box[2])]

# Convert the image to LAB color space
lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

# Split the LAB image into L, A, and B channels
l, a, b = cv2.split(lab)

# Apply color thresholding based on the A and B channels
# Adjust the threshold values to capture the damaged regions based on color
color_mask = cv2.inRange(a, 100, 200) & cv2.inRange(b, 100, 200)

# Apply texture analysis using the L channel
# Apply a Gaussian filter to smooth the image
l_blurred = cv2.GaussianBlur(l, (5, 5), 0)

# Compute the gradient magnitude using the Sobel operator on the smoothed L channel
gradient_x = cv2.Sobel(l_blurred, cv2.CV_64F, 1, 0, ksize=3)
gradient_y = cv2.Sobel(l_blurred, cv2.CV_64F, 0, 1, ksize=3)
gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)

# Normalize the gradient magnitude to the range [0, 255]
gradient_magnitude = cv2.normalize(gradient_magnitude, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

# Apply texture thresholding based on the gradient magnitude
# Adjust the threshold value to capture the damaged regions based on texture
texture_mask = cv2.inRange(gradient_magnitude, 50, 255)

# Combine the color and texture masks
mask = cv2.bitwise_and(color_mask, texture_mask)

# Find contours in the mask to detect damaged areas
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Set the threshold area to filter out small or large areas
threshold_area = 1000  # Adjust this value based on your requirements

# Calculate the total area of the bike
bike_area = cv2.countNonZero(mask)

# Assign a value to 'damage_percentage' before using it


# Iterate over the contours and identify the damaged areas
damage_area = 0
for contour in contours:
    # Filter contours based on area or shape properties to detect damaged areas
    area = cv2.contourArea(contour)
    if area > threshold_area:
        # Find the bounding box of the damaged area
        x, y, w, h = cv2.boundingRect(contour)
        
        # Draw a rectangle around the damaged area
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        damage_percentage = 75.0
        # Add the name of the damaged area near its rectangle
        cv2.putText(image, f"Damaged: {damage_percentage:.2f}%", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
        # Increment the damage area
        damage_area += area

# Calculate the percentage of damage
damage_percentage = (damage_area / bike_area) * 100

# Draw the text for the damage percentage on top of the image
cv2.putText(image, f"Damage: {damage_percentage:.2f}%", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)

# Create an image with double width to display both original and damaged images side by side
concatenated_image = np.hstack((original_image, image))

# Display the bike image with the detected damaged areas and the damage percentage
cv2.imshow("Original Image | Bike Damage Detection", concatenated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
